In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chain-of-thought-collection/CoT_collection.json
/kaggle/input/chain-of-thought-collection/flan_collection_subset.json
/kaggle/input/chain-of-thought-collection/flan_collection_russian.json
/kaggle/input/chain-of-thought-collection/flan_collection_korean.json
/kaggle/input/chain-of-thought-collection/CoT_collection_russian.json
/kaggle/input/chain-of-thought-collection/CoT_collection_korean.json
/kaggle/input/chain-of-thought-collection/flan_collection_chinese.json
/kaggle/input/chain-of-thought-collection/CoT_collection_japanese.json
/kaggle/input/chain-of-thought-collection/flan_collection_japanese.json
/kaggle/input/chain-of-thought-collection/CoT_collection_chinese.json
/kaggle/input/chain-of-thought-collection/flan_collection_french.json
/kaggle/input/chain-of-thought-collection/CoT_collection_french.json


# 1️⃣ Install dependencies

In [2]:
!pip install -q transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

# 2️⃣ Imports & Setup

In [3]:
import os
import json
import random
from datasets import Dataset              # no more load_metric
from transformers import (
    GPT2TokenizerFast,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    set_seed,
)

# reproducibility
set_seed(42)

2025-08-11 07:27:35.468402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754897255.633759      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754897255.687255      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 3️⃣ Data Preprocessing

Load the JSON dataset and transform each example into our input/output format.

In [4]:
# Path to your raw JSON file on Kaggle (adjust if necessary)
DATA_PATH = "/kaggle/input/chain-of-thought-collection/CoT_collection.json"  # example path

# Load JSON
with open(DATA_PATH, "r") as f:
    raw = json.load(f)

CAP = 10000  # jitna train/test me use karna hai, utna hi clean hoga

examples = []
seen = set()

def norm(x: str) -> str:
    return " ".join(x.split()) if isinstance(x, str) else ""

import re, unicodedata

# --- helpers: cleaning primitives ---

URL_RE = re.compile(r"""((https?://|www\.)\S+)|(\S+@\S+\.\S+)""", re.IGNORECASE)
HTML_TAG_RE = re.compile(r"<\s*/?\s*[a-zA-Z][^>]*>", re.MULTILINE)
MARKDOWN_RE = re.compile(r"(```.*?```|`[^`]*`|\*\*|__|~~|>\s*)", re.DOTALL)
EMOJI_RE = re.compile(
    "["                                   # basic emoji/pictograph blocks
    "\U0001F300-\U0001F6FF"               # Misc Symbols and Pictographs
    "\U0001F700-\U0001F77F"               # Alchemical Symbols
    "\U0001F780-\U0001F7FF"               # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FAFF"
    "\U00002700-\U000027BF"               # Dingbats
    "\U00002600-\U000026FF"               # Misc symbols
    "]+",
    flags=re.UNICODE,
)

def normalize_quotes_dashes(s: str) -> str:
    # curly quotes to straight; long dashes to hyphen
    s = s.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")
    s = s.replace("—", "-").replace("–", "-")
    return s

def clean_text(s: str) -> str:
    """Light+ cleaning: keep <> for our tags, strip noise but not regular punctuation."""
    if not isinstance(s, str):
        return ""
    s = unicodedata.normalize("NFKC", s)
    # remove zero-width & BOMs
    s = s.replace("\u200b", "").replace("\ufeff", "").replace("\u2060", "")
    # drop non-printable controls (keep whitespace)
    s = "".join(ch for ch in s if (ch.isprintable() or ch in "\n\t "))
    s = normalize_quotes_dashes(s)
    # strip URLs/emails
    s = URL_RE.sub(" ", s)
    # remove markdown/code fences, inline ticks, bold/underline markers, blockquote >
    s = MARKDOWN_RE.sub(" ", s)
    # strip generic HTML/XML tags BUT keep our task tags <REASONING>, </REASONING>, <ANSWER>, </ANSWER>
    s = re.sub(r"</?code>", " ", s, flags=re.IGNORECASE)   # common stray code tag
    # temporarily mask our tags so the generic stripper doesn't remove them
    s = s.replace("<REASONING>", "§REASON_OPEN§").replace("</REASONING>", "§REASON_CLOSE§")
    s = s.replace("<ANSWER>", "§ANS_OPEN§").replace("</ANSWER>", "§ANS_CLOSE§")
    s = HTML_TAG_RE.sub(" ", s)  # remove other tags
    # unmask our tags
    s = s.replace("§REASON_OPEN§", "<REASONING>").replace("§REASON_CLOSE§", "</REASONING>")
    s = s.replace("§ANS_OPEN§", "<ANSWER>").replace("§ANS_CLOSE§", "</ANSWER>")
    # remove emoji/pictographs
    s = EMOJI_RE.sub(" ", s)
    # collapse noisy symbol runs (but keep <> () : / % = & - + ' " )
    s = re.sub(r"[^\w\s.,!?;:/()<>\-+'\"%=&]+", " ", s)
    # tidy whitespace
    s = re.sub(r"[ \t\r\f\v]+", " ", s)
    s = re.sub(r"\s+\n", "\n", s)
    s = re.sub(r"\n\s+", "\n", s)
    return s.strip()

def mostly_non_latin(s: str, thresh: float = 0.6) -> bool:
    """Heuristic: skip if text is mostly non-Latin (dataset me multilingual noise hota hai)."""
    if not s:
        return True
    letters = [ch for ch in s if ch.isalpha()]
    if not letters:
        return False
    latin = sum('LATIN' in unicodedata.name(ch, '') for ch in letters)
    ratio_nonlatin = 1.0 - (latin / max(1, len(letters)))
    return ratio_nonlatin >= thresh

def strip_artifacts(s: str) -> str:
    """Remove common dataset artifacts leaking into rationale/target."""
    s = re.sub(r"\bAnswer\s*:\s*", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"\bFinal\s*Answer\s*:\s*", " ", s, flags=re.IGNORECASE)
    s = s.replace("</code>", " ")
    return s.strip()

# shuffle keys for random selection (clean only what we use)
keys = list(raw.keys())
random.shuffle(keys)

MIN_COT_WC, MAX_COT_WC = 8, 512
MAX_TGT_TOK = 50  # sanity: targets that look like long paragraphs get skipped

for k in keys:
    item = raw[k]

    # essential fields present?
    if not all([
        item.get("prompt") is not None,
        item.get("source") is not None,
        item.get("rationale") is not None,
        item.get("target") is not None,
    ]):
        continue

    # clean + normalize
    prompt = clean_text(norm(item.get("prompt", "")))
    src    = clean_text(norm(item.get("source", "")))
    cot    = clean_text(norm(item.get("rationale", "")))
    tgt    = clean_text(norm(item.get("target", "")))

    # strip common artifacts
    cot = strip_artifacts(cot)
    tgt = strip_artifacts(tgt)

    # drop empties after cleaning
    if not (prompt and src and cot and tgt):
        continue

    # language heuristic (skip mostly non‑Latin to keep distribution simple)
    if mostly_non_latin(src + " " + cot + " " + tgt, thresh=0.7):
        continue

    # rationale length
    wc = len(cot.split())
    if wc < MIN_COT_WC or wc > MAX_COT_WC:
        continue

    # short target sanity: avoid huge essay targets
    if len(tgt.split()) > MAX_TGT_TOK:
        continue

    # duplicate filter
    key_tuple = (src.lower(), tgt.lower(), cot.lower())
    if key_tuple in seen:
        continue
    seen.add(key_tuple)

    # final formatted input/output
    input_text = (
        f"### Prompt: {prompt}\n"
        f"### Context: {src}\n"
        f"### Chain of Thought:\n"
        f"Let's think step by step.\n"
        f"<REASONING>\n"
    )
    output_text = (
        # f"<REASONING>\n{cot}\n</REASONING>\n"
        # f"<ANSWER> {tgt} </ANSWER>"
        f"{cot}\n</REASONING>\n"
        f"Based on the reasoning above, the final answer is:\n"
        f"<ANSWER> {tgt} </ANSWER>"
    )

    examples.append({"text": input_text, "label": output_text})

    if len(examples) >= CAP:
        break

ds = Dataset.from_list(examples)
print(f"Built {len(ds)} examples (from {len(raw)} raw).")
print(ds[0])

Built 10000 examples (from 1837928 raw).
{'text': "### Prompt: short_general_knowledge_q\n### Context: Short general knowledge question: what all does stan kroenke own?\n### Chain of Thought:\nLet's think step by step.\n<REASONING>\n", 'label': 'The answer is based on the fact that Stan Kroenke owns the Denver Nuggets. Therefore, the answer should be "Denver Nuggets". </code\n</REASONING>\nBased on the reasoning above, the final answer is:\n<ANSWER> Denver Nuggets </ANSWER>'}


### prompt → target pairs

In [5]:
# === Preview a single formatted training pair (for the assignment) ===
sample_io = ds[0]
print("\n--- Training Pair Preview ---")
print("Input:\n", sample_io["text"])
print("\nOutput:\n", sample_io["label"])


--- Training Pair Preview ---
Input:
 ### Prompt: short_general_knowledge_q
### Context: Short general knowledge question: what all does stan kroenke own?
### Chain of Thought:
Let's think step by step.
<REASONING>


Output:
 The answer is based on the fact that Stan Kroenke owns the Denver Nuggets. Therefore, the answer should be "Denver Nuggets". </code
</REASONING>
Based on the reasoning above, the final answer is:
<ANSWER> Denver Nuggets </ANSWER>


# 4️⃣ Train/Test Split

In [6]:
ds = ds.train_test_split(test_size=0.2, seed=42)
train_ds = ds["train"]
eval_ds  = ds["test"]
print(f"Train: {len(train_ds)} examples, Eval: {len(eval_ds)} examples")

Train: 8000 examples, Eval: 2000 examples


# 5️⃣ Tokenization

In [7]:
import torch, transformers
print("torch:", torch.__version__, "| transformers:", transformers.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    print("GPU:", torch.cuda.get_device_name(0))


torch: 2.6.0+cu124 | transformers: 4.52.4
cuda available: True
GPU: Tesla P100-PCIE-16GB


In [8]:
# Load tokenizer & model
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

try:
    model.gradient_checkpointing_enable()
except Exception:
    model.config.gradient_checkpointing = True

print("using device:", next(model.parameters()).device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

using device: cuda:0


In [9]:
# add special tokens for reasoning/answer sections
specials = {"additional_special_tokens": ["<REASONING>", "</REASONING>", "<ANSWER>", "</ANSWER>"]}
tokenizer.add_special_tokens(specials)
model.resize_token_embeddings(len(tokenizer))
# Tokenize function

def tokenize(batch):
    # batch["text"] and batch["label"] are lists when batched=True
    texts = batch["text"]
    lbls  = batch["label"]
    full  = [t + l for t, l in zip(texts, lbls)]

    enc = tokenizer(
        full,
        truncation=True,
        max_length=512,
        padding="max_length",
    )

    # prompt lengths per example (for masking)
    prompt_ids_list = tokenizer(texts, add_special_tokens=True)["input_ids"]

    labels = []
    for i, ids in enumerate(enc["input_ids"]):
        lab = ids.copy()
        cutoff = min(len(prompt_ids_list[i]), len(lab))
        if cutoff > 0:
            lab[:cutoff] = [-100] * cutoff
        labels.append(lab)
    enc["labels"] = labels
    return enc

# Apply
train_tok = train_ds.map(tokenize, remove_columns=train_ds.column_names, batched=True)
eval_tok  = eval_ds.map(tokenize,  remove_columns=eval_ds.column_names,  batched=True)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1263 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# 6️⃣ Data Collator & Training Arguments

In [10]:
# 6️⃣ Data Collator & Training Arguments (updated for logging)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# keep a small dict of key training settings for the write-up
train_settings = {
    "model": "gpt2-medium",
    "train_examples": len(train_ds),
    "eval_examples": len(eval_ds),
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 4,
    "num_train_epochs": 8,
    "learning_rate": 2e-5,
    "weight_decay": 0.1,
    "fp16": True,
}


training_args = TrainingArguments(
    output_dir="gpt2_cot_finetuned",
    do_train=True,
    do_eval=True,

    # logging
    logging_dir="runs",            # where to write TensorBoard logs
    logging_steps=20,              # print/log every 20 training steps
    report_to="tensorboard",       # send logs to TensorBoard

    # progress bar
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.1,
    save_total_limit=2,
    fp16=True,                     # mixed precision

    # ensure the tqdm bar is visible
    disable_tqdm=False,
)

# 7️⃣ Trainer Initialization & Training

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=eval_tok,
    data_collator=data_collator,
)
# trainer.train()
# Manual early‐stopping loop

best_loss = float("inf")
patience = 2
wait = 0
max_epochs = int(training_args.num_train_epochs)

for epoch in range(1, max_epochs + 1):
    print(f"\n===== Epoch {epoch}/{max_epochs} =====")
    trainer.train(resume_from_checkpoint=(epoch > 1))
    metrics = trainer.evaluate()
    val_loss = metrics.get("eval_loss")
    print(f"Epoch {epoch} → eval_loss = {val_loss:.4f}" if val_loss is not None else "No eval_loss")

    if val_loss is not None and val_loss < best_loss:
        best_loss = val_loss
        wait = 0
        print("  ✅ New best — saving checkpoint.")
        trainer.save_model("best-model")
    else:
        wait += 1
        print(f"  No improvement for {wait} epoch(s).")
        if wait >= patience:
            print(f"⏹ Early stop (patience={patience}).")
            break
            
# Save final model & tokenizer
trainer.save_model("gpt2_cot_final")
tokenizer.save_pretrained("gpt2_cot_final")

# -- Save a tiny Model Card / README for submission --
with open("gpt2_cot_final/README.md", "w") as f:
    f.write("# GPT-2 CoT Fine-tune\n")
    f.write("- Base model: gpt2\n")
    f.write(f"- Train examples: {len(train_ds)}\n")
    f.write(f"- Eval examples: {len(eval_ds)}\n")
    f.write(f"- LR: {train_settings['learning_rate']}, Epochs: {train_settings['num_train_epochs']}, BS: {train_settings['per_device_train_batch_size']}\n")
    f.write("- Data: Chain-of-Thought Collection (Kaggle). For research/educational use.\n")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...



===== Epoch 1/3 =====


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,3.251400
40,2.833900
60,2.728700
80,2.578300
100,2.513600
120,2.350300
140,2.378200
160,2.520800
180,2.375000
200,2.333800


Epoch 1 → eval_loss = 1.7020
  ✅ New best — saving checkpoint.

===== Epoch 2/3 =====


...

Step,Training Loss


Epoch 2 → eval_loss = 1.7020
  No improvement for 1 epoch(s).

===== Epoch 3/3 =====


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss


Epoch 3 → eval_loss = 1.7020
  No improvement for 2 epoch(s).
⏹ Early stop (patience=2).


In [12]:
# install tensorboard if needed
!pip install -q tensorboard

# load the tensorboard extension
%load_ext tensorboard

# launch it pointing at our logs folder
%tensorboard --logdir runs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<IPython.core.display.Javascript object>

# 8️⃣ Inference & Qualitative Evaluation

Generate CoT outputs on some unseen examples.

In [13]:
import random

def safe_sample(dataset, k=10, seed=None):
    """
    Returns up to k random samples from dataset.
    Will return fewer if dataset is smaller.
    """
    if seed is not None:
        random.seed(seed)
    dataset_size = len(dataset)
    if dataset_size <= k:
        return list(dataset)
    return random.sample(list(dataset), k)

In [14]:
def parse_prompt_source(text):
    """
    Extracts (prompt, source) from the combined text field.
    Adjust splitting logic to match your dataset format.
    """
    # Example if your text is in format: "### Prompt: ...\n### Context: ..."
    prompt, source = "", text
    if "### Prompt:" in text and "### Context:" in text:
        parts = text.split("### Prompt:", 1)[1].split("### Context:", 1)
        prompt = parts[0].strip()
        source = parts[1].strip()
    return prompt, source

In [15]:
import re
import string

def normalize_text(s):
    """Lowercase, remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        return ''.join(ch for ch in text if ch not in string.punctuation)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match(prediction, ground_truth):
    """True if normalized prediction exactly matches normalized ground truth."""
    return normalize_text(prediction) == normalize_text(ground_truth)

def f1_token(prediction, ground_truth):
    """F1 score between prediction and ground truth based on token overlap."""
    pred_tokens = normalize_text(prediction).split()
    gt_tokens = normalize_text(ground_truth).split()
    common = set(pred_tokens) & set(gt_tokens)
    num_same = sum(min(pred_tokens.count(tok), gt_tokens.count(tok)) for tok in common)

    if len(pred_tokens) == 0 or len(gt_tokens) == 0:
        return int(pred_tokens == gt_tokens)
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    return (2 * precision * recall) / (precision + recall)

In [16]:
# ========= 2‑stage generator: first rationale, then answer, with tag EOS =========

def _ids_len(x): 
    return x["input_ids"].shape[-1]

def _decode_new(out_ids, in_len):
    new_tokens = out_ids[0][in_len:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

# resolve special token ids (fallback if not found)
REASON_OPEN = "<REASONING>"
REASON_CLOSE = "</REASONING>"
ANS_OPEN    = "<ANSWER>"
ANS_CLOSE   = "</ANSWER>"

REASON_CLOSE_ID = tokenizer.convert_tokens_to_ids(REASON_CLOSE)
ANS_CLOSE_ID    = tokenizer.convert_tokens_to_ids(ANS_CLOSE)

USE_REASON_EOS  = REASON_CLOSE_ID != tokenizer.unk_token_id and REASON_CLOSE_ID != -1
USE_ANS_EOS     = ANS_CLOSE_ID    != tokenizer.unk_token_id and ANS_CLOSE_ID    != -1

@torch.no_grad()
def generate_rationale_then_answer(source: str, prompt: str = ""):
    """
    Stage 1: generate rationale until </REASONING>
    Stage 2: generate answer until </ANSWER>
    Returns: rationale_text, answer_text, raw_full_text
    """
    # ---------- Stage 1: RATIONALE ----------
    prefix_reason = (
        f"### Prompt: {prompt}\n"
        f"### Context: {source}\n"
        f"### Chain of Thought:\n"
        f"Let's think step by step.\n"
        f"{REASON_OPEN}\n"
    )
    inp1 = tokenizer(prefix_reason, return_tensors="pt", truncation=True, max_length=900).to(model.device)
    headroom1 = max(32, 1024 - _ids_len(inp1) - 1)

    out1 = model.generate(
        **inp1,
        max_new_tokens=min(256, headroom1),
        min_new_tokens=min(64, headroom1),
        do_sample=False,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=(REASON_CLOSE_ID if USE_REASON_EOS else tokenizer.eos_token_id),
    )
    rationale_segment = _decode_new(out1, _ids_len(inp1))
    # ensure we end with closing tag text (for cleanliness)
    if REASON_CLOSE not in rationale_segment:
        rationale_segment = rationale_segment.split("###")[0].strip()
        rationale_segment = rationale_segment + f"\n{REASON_CLOSE}"

    # ---------- Stage 2: ANSWER ----------
    # prefix_answer = prefix_reason + rationale_segment + f"\n{ANS_OPEN} "
       # Explicitly tie the answer to the generated reasoning
    prefix_answer = (
        prefix_reason
        + rationale_segment
        + "\nBased on the reasoning above, the final answer is:\n"
        + f"{ANS_OPEN} "
    )
    inp2 = tokenizer(prefix_answer, return_tensors="pt", truncation=True, max_length=1000).to(model.device)
    headroom2 = max(16, 1024 - _ids_len(inp2) - 1)

    out2 = model.generate(
        **inp2,
        max_new_tokens=min(64, headroom2),
        min_new_tokens=min(8, headroom2),
        do_sample=False,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=(ANS_CLOSE_ID if USE_ANS_EOS else tokenizer.eos_token_id),
    )
    answer_segment = _decode_new(out2, _ids_len(inp2))
    if ANS_CLOSE not in answer_segment:
        # trim at accidental section headers if any
        answer_segment = answer_segment.split("###")[0].strip()
        answer_segment = answer_segment + f" {ANS_CLOSE}"

    # ---------- Compose clean strings ----------
    # strip tags for display (keeping content)
    def between(txt, open_tag, close_tag):
        if open_tag in txt and close_tag in txt:
            return txt.split(open_tag, 1)[1].split(close_tag, 1)[0].strip()
        return txt.strip()

    rationale_clean = between(rationale_segment, REASON_OPEN, REASON_CLOSE)
    answer_clean    = between(answer_segment, ANS_OPEN, ANS_CLOSE)

    full_text = prefix_reason + rationale_segment + "\n" + ANS_OPEN + " " + answer_clean + " " + ANS_CLOSE
    return rationale_clean, answer_clean, full_text

# ================= run & display with the new 2‑stage generator =================

sampled = safe_sample(eval_ds, k=25, seed=42)
print(f"[info] evaluating on {len(sampled)} examples (eval_ds size = {len(eval_ds)})")

pred_answers, gold_answers = [], []

for ex in sampled:
    prompt, source = parse_prompt_source(ex["text"])
    rationale, answer, raw_full = generate_rationale_then_answer(source, prompt)
    gold_ans = ex["label"]
    # gold answer text (strip tags if present)
    gold_ans_clean = gold_ans
    if "<ANSWER>" in gold_ans_clean:
        gold_ans_clean = gold_ans_clean.split("<ANSWER>", 1)[1].split("</ANSWER>", 1)[0].strip()

    pred_answers.append(answer)
    gold_answers.append(gold_ans_clean)

    print("─" * 80)
    print("SOURCE:", source)
    print("\nMODEL RAW OUTPUT:\n", raw_full)
    print("\nPARSED RATIONALE:\n", rationale if rationale else "[none]")
    print("PARSED ANSWER:\n", answer if answer else "[none]")
    print("\nGROUND TRUTH RATIONALE + ANSWER:\n", ex["label"])

# -- Build a small qualitative table for 10–20 samples --
import pandas as pd
rows = []
for i, ex in enumerate(sampled):
    # recompute clean gold answer
    gold = ex["label"]
    if "<ANSWER>" in gold:
        gold = gold.split("<ANSWER>", 1)[1].split("</ANSWER>", 1)[0].strip()
    rows.append({
        "idx": i,
        "coherent_rationale": "",    # fill manually: yes/no
        "grounded_in_context": "",   # fill manually: yes/no
        "answer_correct": "",        # fill manually: yes/no (compare to gold)
    })
qual_df = pd.DataFrame(rows)
print("\n[Qualitative checklist — mark yes/no]")
print(qual_df.head(10).to_string(index=False))

# --- quick metrics (same as before) ---
if pred_answers and gold_answers:
    em = sum(exact_match(p, g) for p, g in zip(pred_answers, gold_answers)) / len(pred_answers)
    f1 = sum(f1_token(p, g)     for p, g in zip(pred_answers, gold_answers)) / len(pred_answers)
    print("\n== Quick Answer Metrics on sample ==")
    print(f"Exact Match: {em:.3f} | F1 (token): {f1:.3f}")
else:
    print("\n[warn] No predictions or gold answers to score.")

[info] evaluating on 25 examples (eval_ds size = 2000)
────────────────────────────────────────────────────────────────────────────────
SOURCE: See the multi-choice question below: Sentence 1: horrible i guess i would try and have somebody at the house as long as they were healthy enough but sometimes they're Sentence 2: It is unhealthy, I would never try those because I care about myself. If the first sentence is true, then is the second sentence true? OPTIONS: - yes - it is not possible to tell - no
### Chain of Thought:
Let's think step by step.
<REASONING>

MODEL RAW OUTPUT:
 ### Prompt: 4
### Context: See the multi-choice question below: Sentence 1: horrible i guess i would try and have somebody at the house as long as they were healthy enough but sometimes they're Sentence 2: It is unhealthy, I would never try those because I care about myself. If the first sentence is true, then is the second sentence true? OPTIONS: - yes - it is not possible to tell - no
### Chain of Thought:
L

# Evaluation metrics + random/custom testing 

In [17]:
# ========= Evaluation metrics + random/custom testing =========
import re
from collections import Counter
import numpy as np
import random
import pandas as pd

# --- small helpers (reuse-friendly) ---

def _norm_text(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r'\b(a|an|the)\b', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def exact_match(pred: str, gold: str) -> int:
    return int(_norm_text(pred) == _norm_text(gold))

def f1_token(pred: str, gold: str) -> float:
    p = _norm_text(pred).split()
    g = _norm_text(gold).split()
    inter = Counter(p) & Counter(g)
    overlap = sum(inter.values())
    if not p or not g:
        return float(p == g)
    prec = overlap / len(p)
    rec  = overlap / len(g)
    return 0.0 if (prec + rec) == 0 else 2 * prec * rec / (prec + rec)

def _lcs(X, Y):
    # longest common subsequence length for ROUGE-L
    m, n = len(X), len(Y)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if X[i] == Y[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    return dp[m][n]

def rouge_l(pred: str, gold: str) -> float:
    # token-level F1 variant of ROUGE-L
    P = _norm_text(pred).split()
    G = _norm_text(gold).split()
    if not P or not G:
        return float(P == G)
    lcs = _lcs(P, G)
    prec = lcs / len(P)
    rec  = lcs / len(G)
    return 0.0 if (prec + rec) == 0 else 2 * prec * rec / (prec + rec)

def jaccard_overlap(a: str, b: str) -> float:
    A = set(_norm_text(a).split())
    B = set(_norm_text(b).split())
    if not A and not B:
        return 1.0
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

def extract_gold_answer(label_text: str) -> str:
    if "<ANSWER>" in label_text and "</ANSWER>" in label_text:
        return label_text.split("<ANSWER>", 1)[1].split("</ANSWER>", 1)[0].strip()
    return label_text.strip()

def extract_gold_rationale(label_text: str) -> str:
    if "<REASONING>" in label_text and "</REASONING>" in label_text:
        return label_text.split("<REASONING>", 1)[1].split("</REASONING>", 1)[0].strip()
    return label_text.strip()

def constrained_label_accuracy(source_text: str, pred_answer: str, gold_answer: str) -> tuple[bool, str]:
    """
    If the task specifies a closed label set, compute accuracy; else return (None, '').
    """
    s = source_text.lower()
    pa = pred_answer.strip().lower()
    ga = gold_answer.strip().lower()

    # hate / not hate
    if "answer using 'hate' or 'not hate'" in s:
        valid = {"hate", "not hate"}
        return (pa in valid and ga in valid and pa == ga, "hate/not_hate")
    # yes/no
    if re.search(r"\boptions:\s*-\s*no\s*-\s*yes\b", s) or "options: - no - yes" in s:
        valid = {"yes", "no"}
        return (pa in valid and ga in valid and pa == ga, "yes/no")
    # textual entailment common labels
    if re.search(r"\b(entails|neutral|contradiction)\b", s):
        valid = {"entails", "neutral", "contradiction"}
        return (pa in valid and ga in valid and pa == ga, "nli")
    # A-E multiple choice
    if re.search(r"\boptions?:.*\b([a-e])\b", s, flags=re.IGNORECASE|re.DOTALL):
        valid = {"a","b","c","d","e"}
        pa1 = pa[:1]
        ga1 = ga[:1]
        return (pa1 in valid and ga1 in valid and pa1 == ga1, "mcq_A-E")

    return (None, "")

# --- main evaluation over a subset of eval_ds ---
def evaluate_model(eval_dataset, k=100, seed=42):
    sampled = eval_dataset.shuffle(seed=seed).select(range(min(k, len(eval_dataset))))
    rows = []

    ans_em_list, ans_f1_list = [], []
    lab_hits, lab_total = 0, 0
    rouge_list, cov_list = [], []

    for ex in sampled:
        prompt, source = parse_prompt_source(ex["text"])
        rat, ans, raw = generate_rationale_then_answer(source, prompt)

        gold_ans = extract_gold_answer(ex["label"])
        gold_rat = extract_gold_rationale(ex["label"])

        # answers
        ans_em_list.append(exact_match(ans, gold_ans))
        ans_f1_list.append(f1_token(ans, gold_ans))

        # label accuracy if constrained
        hit, mode = constrained_label_accuracy(source, ans, gold_ans)
        if hit is not None:
            lab_total += 1
            lab_hits += int(hit)

        # rationales
        rouge_list.append(rouge_l(rat, gold_rat))
        cov_list.append(jaccard_overlap(rat, source))  # how much rationale shares vocab with context

        rows.append({
            "source_snip": source[:100] + ("..." if len(source) > 100 else ""),
            "pred_answer": ans,
            "gold_answer": gold_ans,
            "EM": ans_em_list[-1],
            "F1": round(ans_f1_list[-1], 3),
            "ROUGE-L(rationale)": round(rouge_list[-1], 3),
            "Cov(rationale,context)": round(cov_list[-1], 3),
        })

    # summary
    em = np.mean(ans_em_list) if ans_em_list else 0.0
    f1 = np.mean(ans_f1_list) if ans_f1_list else 0.0
    rouge_mean = np.mean(rouge_list) if rouge_list else 0.0
    cov_mean = np.mean(cov_list) if cov_list else 0.0
    lab_acc = (lab_hits / lab_total) if lab_total > 0 else None

    print(f"\n[Evaluation on {len(sampled)} examples]")
    print(f" Answer EM:  {em:.3f}")
    print(f" Answer F1:  {f1:.3f}")
    if lab_acc is not None:
        print(f" Label Accuracy ({lab_total} constrained cases): {lab_acc:.3f}")
    print(f" Rationale ROUGE-L (vs gold): {rouge_mean:.3f}")
    print(f" Rationale Coverage (vs context Jaccard): {cov_mean:.3f}")

    df = pd.DataFrame(rows)
    # show a small preview table
    try:
        from IPython.display import display
        display(df.head(10))
    except:
        print(df.head(10).to_string(index=False))

    return {
        "em": em, "f1": f1, "label_acc": lab_acc,
        "rougeL": rouge_mean, "coverage": cov_mean,
        "details": df
    }

# --- run evaluation (tweak k to 10–20 for the assignment) ---
_ = evaluate_model(eval_ds, k=15, seed=42)

# --- RANDOM quick check from eval set ---
def random_infer(n=3, seed=None):
    if seed is not None:
        random.seed(seed)
    idxs = random.sample(range(len(eval_ds)), k=min(n, len(eval_ds)))
    for i in idxs:
        ex = eval_ds[i]
        prompt, source = parse_prompt_source(ex["text"])
        rat, ans, raw = generate_rationale_then_answer(source, prompt)
        print("—"*80)
        print(f"[{i}] PROMPT:", prompt or "[none]")
        print("SOURCE:", source)
        print("\nMODEL RATIONALE:\n", rat)
        print("MODEL ANSWER:", ans)
        print("\nGOLD:", extract_gold_answer(ex['label']))

# try it
random_infer(n=3, seed=123)

# --- CUSTOM ad‑hoc inference on your own text ---
def custom_infer(prompt: str, context: str):
    rat, ans, raw = generate_rationale_then_answer(context, prompt)
    print("PROMPT:", prompt or "[none]")
    print("CONTEXT:", context)
    print("\nMODEL RATIONALE:\n", rat)
    print("MODEL ANSWER:", ans)
    return rat, ans

# example:
# custom_infer("classification", "Given a comment, classify it into a 'hate' or 'not hate': 'I hate you'")



[Evaluation on 15 examples]
 Answer EM:  0.000
 Answer F1:  0.070
 Label Accuracy (1 constrained cases): 0.000
 Rationale ROUGE-L (vs gold): 0.284
 Rationale Coverage (vs context Jaccard): 0.175


,source_snip,pred_answer,gold_answer,EM,F1,ROUGE-L(rationale),"Cov(rationale,context)"
0,Suggest a movie title for the following movie ...,Burke Burke Harry romani Harry romano Janet S...,Firewall,0,0.000,0.171,0.087
1,In this task you will be given a claim and a p...,undermine n neutral neutral </ANSWER>,undermine,0,0.333,0.412,0.234
2,Question: Tim discovered that his polished flo...,quilted polished </ANSWER>,quilt,0,0.000,0.222,0.222
3,In this task you will be given a text passage ...,Ummmaysad Ummmmaysad n Ummas </ANSWER>,History,0,0.000,0.451,0.136
4,When enjoying flowers in a recreation area run...,surface \n \n\nBased </ANSWER>,state park,0,0.000,0.353,0.244
5,Write the conversation response. DIALOG: Where...,"Hayns , Woody , Pearson and Pearson Pearson ...",What health problems did Gregg Allman have dur...,0,0.039,0.125,0.209
6,What is the solution? Solve -212514 + 213261 =...,</ANSWER>,-3,0,0.000,0.354,0.163
7,You are given a sentence from a conversation b...,Busses Bus Ferry \n Ferry\n Buses is the...,RideSharing,0,0.000,0.361,0.192
8,Article: Every family with kids has seen its s...,No n No No \n No Yes </ANSWER>,No,0,0.250,0.130,0.026
9,Question: How many doctors saw Frédéric by the...,no n no The answer is\n no yes n no </ANSWER>,yes,0,0.182,0.311,0.235


————————————————————————————————————————————————————————————————————————————————
[107] PROMPT: Write a multi-choice question (options given)
SOURCE: Write a multi-choice question for the following article, with the given choices and answer: Article: The "melting pot" in American cuisine(. ) is a myth, not terribly unlike the idea of a melting pot of American culture, notes chef Dan Barber. "Most cultures don't think about their cuisine in such monolithic terms," he says. "French, Mexican, Chinese, and Italian cuisines each consist of dozens of distinct regional foods. And I think 'American' cuisine is moving in the same direction, becoming more localized, not globalized. " American cuisine is shaped by the natural wealth of the country. Having never faced agricultural hardship, Americans don't have to rely on rotating crops, such as the Japanese, whose food culture now showcases buckwheat alongside rice, or the Indians, or the French and Italians, who feature beans alongside wheat. "Th

In [18]:
custom_infer("short_general_knowledge_q", "what is 7-9")

PROMPT: short_general_knowledge_q
CONTEXT: what is 7-9

MODEL RATIONALE:
 The answer is 7.

Based on the reasoning above, the final answer is:
 7  7  7
 The answer is therefore:
 7  7
 The answer to this question is therefore "7".
 What is the answer to that question?  What is "What is " what is the " answer to " that question?"
</REASONING>
MODEL ANSWER: What's the answer?  
 What's " what's the " what are the answer?"
 What's what's what are they answer?
 The answer here is "what are they are answer?" 
 The answer for this question?

Based The final answer for the </ANSWER>


('The answer is 7.\n\nBased on the reasoning above, the final answer is:\n 7  7  7\n The answer is therefore:\n 7  7\n The answer to this question is therefore "7".\n What is the answer to that question?  What is "What is " what is the " answer to " that question?"\n</REASONING>',
 'What\'s the answer?  \n What\'s " what\'s the " what are the answer?"\n What\'s what\'s what are they answer?\n The answer here is "what are they are answer?" \n The answer for this question?\n\nBased The final answer for the </ANSWER>')

In [19]:
custom_infer("short_general_knowledge_q", "what is 2+4")

PROMPT: short_general_knowledge_q
CONTEXT: what is 2+4

MODEL RATIONALE:
 The answer is 2 + 4. The question asks what is the answer to this question. The answer is "2 + 4" because it is a general knowledge question.

Based on the reasoning above, the final answer is:
 2 +  2  
Based
Based upon the reasoning below, the answer is
 3  3
</REASONING>
MODEL ANSWER: n  
   3 </ANSWER>


('The answer is 2 + 4. The question asks what is the answer to this question. The answer is "2 + 4" because it is a general knowledge question.\n\nBased on the reasoning above, the final answer is:\n 2 +  2  \nBased\nBased upon the reasoning below, the answer is\n 3  3\n</REASONING>',
 'n  \n   3 </ANSWER>')